<a href="https://colab.research.google.com/github/kxpil09/Twitter-Scraper-Test-VScale-LLP-/blob/main/coding_x_twitterscrapertest_bitgoro__final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Problem Statement**



1.   We want to scrape text from given list of tweets
2.   We want to save output from extracted text to Output worksheet
3.   you can use any libraries you want
4.   This should not take any more then 4 hours to complete.

### Sample Input (Form 1 Responses Worksheet) : 
https://docs.google.com/spreadsheets/d/1slvCX233_Qwqw1JPENxSL7qZzMLFwUmPbvFC6p7YJ_Q/edit#gid=1346770370

### Sample Output (Output Worksheet): 
https://docs.google.com/spreadsheets/d/1slvCX233_Qwqw1JPENxSL7qZzMLFwUmPbvFC6p7YJ_Q/edit#gid=1346770370


## **Essential Learning Material**

https://we.tl/t-fT1bUrD0OU


In [3]:
!pip install tweepy
!pip install gspread==4.0.0
!pip install oauth2client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gspread
    Found existing installation: gspread 3.4.2
    Uninstalling gspread-3.4.2:
      Successfully uninstalled gspread-3.4.2


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
!pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# importing the required libraries
import gspread
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from oauth2client.service_account import ServiceAccountCredentials
import requests

In [3]:
credential_url = "https://gist.githubusercontent.com/vrushangdev/e26987231a1e8517b6b7f3487f74d6d0/raw/9339c09c2298a3b91fc743c67ee880ce79e1c5e7/gsheet_creds.json"

In [4]:
response = requests.get(credential_url)

In [5]:
try:
  cred_file = open('file.json','w')
  cred_file.write(response.text)
  cred_file.close()
except Exception as e:
  print(e)
finally:
  cred_file.close()

In [6]:
import gspread

In [7]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials


In [8]:
gc = gspread.service_account(filename='/content/file.json')

In [10]:
gsheet = gc.open_by_url("https://docs.google.com/spreadsheets/d/1slvCX233_Qwqw1JPENxSL7qZzMLFwUmPbvFC6p7YJ_Q")


In [11]:
work_sheet = gsheet.worksheets()[0]

In [12]:
twitter_links = work_sheet.col_values(5)

In [ ]:
twitter_links

In [ ]:
print(len(twitter_links))

In [15]:
!pip install git+https://github.com/JustAnotherArchivist/snscrape.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/JustAnotherArchivist/snscrape.git to /tmp/pip-req-build-7tln8xie
  Running command git clone --filter=blob:none --quiet https://github.com/JustAnotherArchivist/snscrape.git /tmp/pip-req-build-7tln8xie
  Resolved https://github.com/JustAnotherArchivist/snscrape.git to commit 786815dd05681e2421cd03aa9acf5ab5c773bce9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import snscrape.modules.twitter as sntwitter
import pandas as pd

# set up Google Sheets credentials
scope = ["https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("file.json", scope)
client = gspread.authorize(creds)

# Get the first two Twitter links from the Google Sheet
gc = gspread.service_account(filename='/content/file.json')
gsheet = gc.open_by_url("https://docs.google.com/spreadsheets/d/1slvCX233_Qwqw1JPENxSL7qZzMLFwUmPbvFC6p7YJ_Q")

work_sheet = gsheet.worksheets()[0]
twitter_links = list(set(work_sheet.col_values(5)[1:2]))

# Scrape Twitter for each link and save the results to a CSV file

data = []
for link in twitter_links:
    tweet_id = link.split("/")[-1]
    try:
        # scrape the influencer tweet
        influencer_tweet = next(sntwitter.TwitterSearchScraper(f"since_id:{tweet_id}").get_items())

        # extract the data
        data.append({
            "time_stamp": influencer_tweet.date.strftime("%Y-%m-%d %H:%M:%S"),
            "influencer": influencer_tweet.user.username,
            "promoter": "",
            "influencer_tweet": influencer_tweet.url,
            "promotion_tweet": link,
            "influencer_tweet_likes": influencer_tweet.likeCount,
            "promoter_tweet_likes": "",
            "influencer_tweet_text": influencer_tweet.rawContent,
            "promoter_tweet_text": ""
        })

        # scrape the promoter tweet
        promoter_tweets = sntwitter.TwitterSearchScraper(f"from:{influencer_tweet.user.username} {link}").get_items()
        for promoter_tweet in promoter_tweets:
            if link in promoter_tweet.rawContent:
                data.append({
                    "time_stamp": promoter_tweet.date.strftime("%Y-%m-%d %H:%M:%S"),
                    "influencer": "",
                    "promoter": promoter_tweet.user.username,
                    "influencer_tweet": influencer_tweet.url,
                    "promotion_tweet": link,
                    "influencer_tweet_likes": influencer_tweet.likeCount,
                    "promoter_tweet_likes": promoter_tweet.likeCount,
                    "influencer_tweet_text": influencer_tweet.rawContent,
                    "promoter_tweet_text": promoter_tweet.rawContent
                })
                break  # only add the first promoter tweet
    except StopIteration:
        # add empty data if no tweet found
        data.append({
            "time_stamp": "",
            "influencer": "",
            "promoter": "",
            "influencer_tweet": "",
            "promotion_tweet": link,
            "influencer_tweet_likes": "",
            "promoter_tweet_likes": "",
            "influencer_tweet_text": "",
            "promoter_tweet_text": ""
        })

# create a pandas dataframe from the data
df = pd.DataFrame(data, columns=["time_stamp", "influencer", "promoter", "influencer_tweet", "promotion_tweet", "influencer_tweet_likes", "promoter_tweet_likes", "influencer_tweet_text", "promoter_tweet_text"])

# save the dataframe to a CSV file
df.to_csv("output.csv", index=False)



In [25]:
df.head()

,time_stamp,influencer,promoter,influencer_tweet,promotion_tweet,influencer_tweet_likes,promoter_tweet_likes,influencer_tweet_text,promoter_tweet_text
0,2023-05-26 15:07:28,treecIimbers,,https://twitter.com/treecIimbers/status/166211...,https://twitter.com/psalmcrypt/status/14424604...,0,,@conicalcanines @inwond3rlandd She’s 19 with a...,


***I only used 1 url in the input because adding all urls takes a lot of time and causes Google Collab servers to frequently hang. This code merely illustrates how the necessary task was implemented.Changes can be made to "twitter_links = list(set(work_sheet.col_values(5)[1:2])") to change the number of input urls.*** 


---




